# Tests for `predictor.utils`

## How to use:

Check your current kernel, if it's not the project's venv, change it:

* Click the top right button to change the kernel to your project's venv (e.g. `'mobilidade_rio_api' (Python 3.10.8)` ([learn more](https://code.visualstudio.com/docs/python/jupyter-support-py#_connect-to-a-remote-jupyter-server)).

# The code

1 - Config Django for use

In [1]:
import os
import sys
# set relative path to server's root
sys.path.append('../..')
# set variables
os.environ["DJANGO_SETTINGS_MODULE"] = "mobilidade_rio.settings.native"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
# start django
import django
django.setup()

Minimal test

In [2]:
from mobilidade_rio.pontos.models import *

queryset = Stops.objects.all()[0:2]
print(queryset)

<QuerySet [<Stops: Stops object (1001O00006C0)>, <Stops: Stops object (1002O00001C0)>]>


## utils.py

### 2 - Imports

In [2]:
import pandas as pd
# django
from mobilidade_rio.utils import query_utils as qu
from mobilidade_rio.utils import jupyter_utils as ju
from mobilidade_rio.utils import django_utils as du
from mobilidade_rio.pontos.models import *
from mobilidade_rio.predictor.models import *
# query
from django.db import connection
cur = connection.cursor() 
cols = du.get_cols((StopTimes, Shapes))
print(cols)

model <class 'django.db.models.base.ModelBase'>
ret <class 'list'>
model <class 'django.db.models.base.ModelBase'>
ret <class 'list'>
['id', 'trip_id', 'stop_sequence', 'stop_id', 'arrival_time', 'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type', 'continuous_pickup', 'continuous_drop_off', 'shape_dist_traveled', 'timepoint', 'shape_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence']


Fetch data from the database to memory

It takes 10s

In [5]:
# Convert to dataframe
df_trips = pd.DataFrame.from_records(Trips.objects.all().order_by("pk").values())
df_stops = pd.DataFrame.from_records(Stops.objects.all().order_by("pk").values())
df_shapes = pd.DataFrame.from_records(Shapes.objects.all().order_by("pk").values())
df_stoptimes = pd.DataFrame.from_records(StopTimes.objects.all().order_by("pk").values())
df_routes = pd.DataFrame.from_records(Routes.objects.all().order_by("pk").values())
# queryset = Stops.objects.all().order_by("stop_id")

# print
# display(df_stops)

Log print

In [6]:
print("Trips")
# print as title
display(df_trips.head(2))
print("Shapes")
display(df_shapes)
print("Stops")
display(df_stops.head(2))
print("Stoptiems")
display(df_stoptimes.head(2))

Trips


,route_id_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,wheelchair_accessible,bikes_allowed
0,btwr,U,0373fd52-e961-45ea-afb2-5959b62d11e3,None,None,0.0,None,bosw,None,None
1,O0825AAA0A,U,175b00bb-a3ea-468b-93a2-bf6c9ff8aced,Jesuítas,825,NaN,None,qi1g,None,None


Shapes


,id,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled
0,6322834,2qdl,-22.868396,-43.477524,0,0.000000
1,6322835,2qdl,-22.868144,-43.477523,1,28.117420
2,6322836,2qdl,-22.868044,-43.477523,2,39.163505
3,6322837,2qdl,-22.868035,-43.477523,3,40.167699
4,6322838,2qdl,-22.867918,-43.477522,4,53.222113
...,...,...,...,...,...,...
890114,7212948,O0702AAA0AVDU01,-22.900721,-43.346640,293,8208.670757
890115,7212949,O0702AAA0AVDU01,-22.900728,-43.346646,294,8209.672589
890116,7212950,O0702AAA0AVDU01,-22.901022,-43.346904,295,8251.749105
890117,7212951,O0702AAA0AVDU01,-22.901029,-43.346910,296,8252.750753


Stops


,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station_id,stop_timezone,wheelchair_boarding,level_id,platform_code
0,0c7g,None,Terminal Paulo da Portela - Ônibus Convencionais,None,-22.878399,-43.336129,None,None,0.0,None,None,None,None,None
1,0nk7,None,Estrada Aterrado do Leme,None,-22.885719,-43.669292,None,None,NaN,None,None,None,None,None


Stoptiems


,id,trip_id_id,stop_sequence,stop_id_id,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint
0,2210600,O0663AAA0ACDU01,88,3063O00023C0,07:46:28,07:46:28,None,0.0,0.0,NaN,NaN,39925.0,0.0
1,2210601,O0663AAA0ACDU01,87,3063O00022C0,07:45:46,07:45:46,None,0.0,0.0,NaN,NaN,39681.0,0.0


### ❌ shapes with stops based on UFRJ

#### Reason for cancellation

* The function requires a `df_stops` but it is a pre rendered table that not exists in server data.
* It will me improductive to compare query with a function using pre rendered table.

In [7]:
def pd_shapes_stops_routes(df_shapes, df_stops):
    """
    For each shape, get the stops and the route
    """

    df_shapes = df_shapes.copy()
    shape_id = df_shapes.iloc[0].shape_id

    sel_df_stops = df_stops[df_stops.trip_id_id == shape_id].rename(
        columns={
            "stop_lat": "latitude",
            "stop_lon": "longitude"
        }
    )

    # pd.concat is 
    df_shapes_stops = pd.concat([
        sel_df_stops[["trip_id_id", "stop_id", "stop_name", "latitude", "longitude", "shape_dist_traveled", "stop_sequence", "route_id", "route_short_name", "route_long_name"]],
        df_shapes[["shape_id", "latitude", "longitude", "shape_dist_traveled"]].rename(columns={"shape_id": "trip_id_id"})
    ], axis=0).sort_values(["shape_dist_traveled", "stop_id"])
        
    df_shapes_stops[["previous_stop_id", "previous_stop_name"]] = df_shapes_stops[["stop_id", "stop_name"]].ffill()
    df_shapes_stops[["next_stop_id", "next_stop_name"]] = df_shapes_stops[["stop_id", "stop_name"]].bfill().shift(-1)
    
    df_shapes_stops = df_shapes_stops[~(df_shapes_stops["next_stop_id"].isna() & df_shapes_stops["stop_id"].isna())]        # remove os shapes nulos
    df_shapes_stops = df_shapes_stops[~(df_shapes_stops["previous_stop_id"].isna() & df_shapes_stops["stop_id"].isna())]    # remove os shapes nulos
        
    return df_shapes_stops

# use function
# df = pd_shapes_stops_routes(df_shapes, df_stops)
# display(df)

Base test

In [8]:
# # connect

# conn = psycopg2.connect("dbname=postgres user=postgres password=postgres port=5432")
# cur = connection.cursor()

# # query

# q_col_in = (
#     qu.q_limit(
#     qu.q_col_in(
#         select=["id","stop_sequence"],
#         from_target="select * from pontos_stoptimes",
#         where_col_in={"stop_id_id": ["4128O00169P0"]},
#         order_by="stop_sequence",
#         target_is_query=True
#     ),
#     limit=30,
#     )
# )

# q_unique_cols = (
#     qu.q_cols_match_all(
#         table="pontos_stoptimes",
#         unique_cols=["trip_id_id","stop_id_id"],
#         select=["*"],
#         col_in={"stop_id_id": ["4128O00122C0","2028O00023C0"]},
#         col_match_all=["trip_id_id"],
#         # q_conditions="stop_id_id IN ('4128O00122C0')",
#     )
# )

# q_raw = """
# SELECT id, stop_id_id, trip_id_id FROM pontos_stoptimes LIMIT 10
# """

# # execute
# q = q_unique_cols
# ju.print_query(q)
# ju.plot_query(cur, q)

## Test raw sql

### ✅ shapes with stops v2

O mesmo que o v1, mas com opção de filtrar distância geográfica.

In [29]:
# shapes_stops v2
q_shapes_stops_tst = """
SELECT 
    id, trip_id, stop_id, shape_id,
    shape_pt_lat, shape_pt_lon, stop_lat, stop_lon
FROM (
    SELECT DISTINCT ON (id) * FROM (
        SELECT
            pontos_stoptimes.id as id,
            trip_id,
            trip_short_name,
            pontos_stoptimes.stop_sequence,
            pontos_stops.stop_id,
            pontos_stoptimes.shape_dist_traveled,
            pontos_shapes.shape_id,
            shape_pt_sequence,
            stop_name,
            shape_pt_lat,
            shape_pt_lon,
            stop_lat,
            stop_lon,
            pontos_routes.route_id,
            route_short_name,
            route_long_name,

            --CAST(SQRT(POW(CAST(shape_pt_lat AS DECIMAL(10,6)) - CAST(stop_lat AS DECIMAL(10,6)),2)
            -- + POW(CAST(shape_pt_lon AS DECIMAL(10,6)) - CAST(stop_lon AS DECIMAL(10,6)),2))
            --    AS DECIMAL(10,6)) AS distance

            ST_Distance(
                ST_MakePoint(shape_pt_lon, shape_pt_lat)::geography,
                ST_MakePoint(stop_lon, stop_lat)::geography
            ) AS distance

        FROM pontos_stoptimes
            JOIN pontos_trips ON (pontos_trips.trip_id = pontos_stoptimes.trip_id_id
            AND pontos_stoptimes.stop_id_id IN ('1001O00006C0', '1003O00006C0'))
            JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
            JOIN pontos_routes ON pontos_trips.route_id_id = pontos_routes.route_id
            JOIN pontos_shapes ON pontos_trips.shape_id = pontos_shapes.shape_id
        ) as t1
    ORDER BY id, distance
) as t2
"""

# save to csv
df = pd.read_sql_query(q_shapes_stops_tst, connection)
# df.to_csv("df.csv", index=False)
q = q_shapes_stops_tst
# ju.print_query(q)
ju.plot_query(cur, q)

C:\Users\Rapha\AppData\Local\Temp\ipykernel_34804\3312138490.py:47: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q_shapes_stops_tst, connection)


len: 10


id,trip_id,stop_id,shape_id,shape_pt_lat,shape_pt_lon,stop_lat,stop_lon
2300587,O0104AAA0AVDU03,1003O00006C0,O0104AAA0AVDU03,-22.897663,-43.202313,-22.897653,-43.202279
2310237,O0384AAA0AIDU02,1001O00006C0,O0384AAA0AIDU01,-22.897331,-43.18731,-22.89731,-43.18737
2322638,O0265AAA0AIDU01,1001O00006C0,7ca9f20c-100a-41f7-9950-ead83d3e1031,-22.897329,-43.187309,-22.89731,-43.18737
2322834,O0355AAA0AIDU02,1001O00006C0,O0355AAA0AIDU02,-22.897333,-43.18731,-22.89731,-43.18737
2328848,E2145AAA0AVDU01,1003O00006C0,E2145AAA0AVDU01,-22.89767,-43.20231,-22.897653,-43.202279
2329320,O0041CAA0AIDU01,1001O00006C0,O0041CAA0AIDU01,-22.897333,-43.187311,-22.89731,-43.18737
2334766,E2018AAA0AIDU01,1001O00006C0,E2018AAA0AIDU01,-22.897332,-43.18731,-22.89731,-43.18737
2334770,E2018AAA0AIDU01,1003O00006C0,E2018AAA0AIDU01,-22.897663,-43.20231,-22.897653,-43.202279
2337205,O0350AAA0AIDU02,1001O00006C0,O0350AAA0AIDU02,-22.897334,-43.187311,-22.89731,-43.18737
2337982,O0385AAA0AIDU01,1001O00006C0,O0385AAA0AIDU01,-22.897333,-43.18731,-22.89731,-43.18737


### ⚠️ (deprecated) shapes with stops v2

The script seems cleaner, but it is not possible to select columns other than those in GROUP BY.

In [19]:
# shapes_stops v2
q_shapes_stops_tst = """
SELECT DISTINCT ON (stop_id, trip_id) * FROM (
    SELECT stop_id, trip_id, t1.shape_id,
    CAST(
        SQRT(POW(CAST(shape_pt_lat AS DECIMAL(10,6)) - CAST(stop_lat AS DECIMAL(10,6)),2)
        + POW(CAST(shape_pt_lon AS DECIMAL(10,6)) - CAST(stop_lon AS DECIMAL(10,6)),2)
    ) AS DECIMAL(10,6)) AS distance

FROM (
    SELECT * FROM pontos_stoptimes
        JOIN pontos_trips ON (
            pontos_trips.trip_id = pontos_stoptimes.trip_id_id
            AND pontos_stoptimes.stop_id_id IN ('1001O00006C0', '1003O00006C0')
        )
        JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
        JOIN pontos_routes ON pontos_trips.route_id_id = pontos_routes.route_id
    ORDER BY pontos_stoptimes.trip_id_id, pontos_stoptimes.stop_sequence
) as t1
JOIN pontos_shapes ON t1.shape_id = pontos_shapes.shape_id
GROUP BY stop_id, trip_id, t1.shape_id, distance
) AS t2
"""

# save to csv
df = pd.read_sql_query(q_shapes_stops_tst, connection)
# df.to_csv("df.csv", index=False)
q = q_shapes_stops_tst
# ju.print_query(q)
ju.plot_query(cur, q)

C:\Users\Rapha\AppData\Local\Temp\ipykernel_28092\664840434.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q_shapes_stops_tst, connection)


len: 10


stop_id,trip_id,shape_id,distance
1001O00006C0,E2018AAA0AIDU01,E2018AAA0AIDU01,0.000064
1001O00006C0,O0041CAA0AIDU01,O0041CAA0AIDU01,0.000063
1001O00006C0,O0265AAA0AIDU01,7ca9f20c-100a-41f7-9950-ead83d3e1031,0.000064
1001O00006C0,O0350AAA0AIDU02,O0350AAA0AIDU02,0.000064
1001O00006C0,O0355AAA0AIDU02,O0355AAA0AIDU02,0.000064
1001O00006C0,O0384AAA0AIDU02,O0384AAA0AIDU01,0.000064
1001O00006C0,O0385AAA0AIDU01,O0385AAA0AIDU01,0.000064
1003O00006C0,E2018AAA0AIDU01,E2018AAA0AIDU01,0.000033
1003O00006C0,E2145AAA0AVDU01,E2145AAA0AVDU01,0.000035
1003O00006C0,O0104AAA0AVDU03,O0104AAA0AVDU03,0.000035


### ✅ 3 - q_stoptimes__in

Function to filter by a list of stops using internal criteria.

Availabe in [`query_django_utils.py`](../utils/query_django_utils.py).

Used by [`pontos.views.py`](../pontos/views.py) > stoptimes.

In [3]:
def q_stoptimes__in(stop_id=None, select=("*"), query=None):
    """
    Filter stoptimes by stop_id or children

    Args:
        query (QuerySet|str): QuerySet to filter
        stop_id (str): stop_id to filter

    Returns:
        query = <queryset>:
            QuerySet filtered by stop_id or children
        query = <str>:
            SQL query filtered by stop_id or children

    """
    # get real col names and stuff
    STOPTIMES_TABLE = StopTimes._meta.db_table
    STOP_ID__STOPTIMES = StopTimes._meta.get_field("stop_id").column
    PARENT_STATION__STOPS = Stops._meta.get_field("parent_station").column

    # validade stop_id
    if stop_id is None:
        stop_id = ()

    # filter location_type
    location_type = Stops.objects.filter(
        stop_id__in=stop_id).values_list("location_type", flat=True)

    # validate query
    raw_filter_used = isinstance(query, django.db.models.query.QuerySet) == False


    # prevent error on searching inexistent stop_id
    # TODO: filter stop_id or children individually
    if len(location_type):
        # validate query
        if query is None:
            query = STOPTIMES_TABLE
        else:
            query = f"({query}) AS {qu.q_random_hash()}"
            
        # if station is parent_station, return children
        if location_type[0] == 1:
            if raw_filter_used:
                query = f"""
                SELECT {','.join(select)}
                FROM {query}
                WHERE {STOP_ID__STOPTIMES} IN (
                    SELECT stop_id FROM pontos_stops
                    WHERE {PARENT_STATION__STOPS} IN ({str(list(stop_id))[1:-1]})
                )
                """
            else:
                query = query.filter(
                    stop_id__in=Stops.objects.filter(
                        parent_station__in=stop_id).values_list("stop_id", flat=True)
                )
        # if first stop has no child (location_type = 0|None), return searched stations
        else:
            if raw_filter_used:
                query = f"""
                SELECT {','.join(select)}
                FROM {query}
                WHERE {STOP_ID__STOPTIMES} IN ({str(list(stop_id))[1:-1]})
                """
            else:
                query = query.filter(stop_id__in=stop_id)
                
    elif query is None:
        query = StopTimes.objects.all().order_by("trip_id")

    return query

# test query vs queryset
q = StopTimes.objects.all().query.__str__()
q = q_stoptimes__in(
    # test params
    ['1001O00006C0', '1003O00006C0'],
    query=q
    )

# print if queryset
if isinstance(q, django.db.models.query.QuerySet):
    df = pd.DataFrame(list(q.values()))
    display(df)

# print if query
else:
    ju.print_query(q)
    ju.plot_query(cur, q)

001 
002                 SELECT *
003                 FROM (SELECT "pontos_stoptimes"."id", "pontos_stoptimes"."trip_id_id", "pontos_stoptimes"."stop_sequence", "pontos_stoptimes"."stop_id_id", "pontos_stoptimes"."arrival_time", "pontos_stoptimes"."departure_time", "pontos_stoptimes"."stop_headsign", "pontos_stoptimes"."pickup_type", "pontos_stoptimes"."drop_off_type", "pontos_stoptimes"."continuous_pickup", "pontos_stoptimes"."continuous_drop_off", "pontos_stoptimes"."shape_dist_traveled", "pontos_stoptimes"."timepoint" FROM "pontos_stoptimes") AS q__25d5ae8834f8bedbbeb1c0c36d055b07
004                 WHERE stop_id_id IN ('1001O00006C0', '1003O00006C0')
005                 
len: 10


id,trip_id_id,stop_sequence,stop_id_id,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint
2337205,O0350AAA0AIDU02,62,1001O00006C0,01:18:25,01:18:25,None,None,None,None,None,22890.872400628974,0
2337982,O0385AAA0AIDU01,73,1001O00006C0,05:39:15,05:39:15,None,0,0,None,None,36595.0,0
2300587,O0104AAA0AVDU03,3,1003O00006C0,00:02:17,00:02:17,None,None,None,None,None,594.0,0
2310237,O0384AAA0AIDU02,85,1001O00006C0,01:48:29,01:48:29,None,None,None,None,None,38083.0,0
2322638,O0265AAA0AIDU01,79,1001O00006C0,01:25:15,01:25:15,None,0,0,None,None,25374.0,0
2322834,O0355AAA0AIDU02,69,1001O00006C0,01:18:12,01:18:12,None,None,None,None,None,27451.0,0
2328848,E2145AAA0AVDU01,14,1003O00006C0,00:20:32,00:20:32,None,None,None,None,None,7817.0,0
2329320,O0041CAA0AIDU01,11,1001O00006C0,05:45:09,05:45:09,None,0,0,None,None,16270.932695400263,0
2334766,E2018AAA0AIDU01,13,1001O00006C0,00:56:57,00:56:57,None,0,0,None,None,18364.691150146442,0
2334770,E2018AAA0AIDU01,9,1003O00006C0,00:50:26,00:50:26,None,0,0,None,None,16265.240298304165,0


### ✅ 4 - stops with previous, next

In [4]:
# add cols previous_stop_id and next_stop_id from pontos_stoptimes
# use stop_sequence to get previous and next stop_id
def get_extra_stops_cols(table_name, return_type="list"):
    ret = [
        f"""
            (
                SELECT stop_id_id
                FROM pontos_stoptimes
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence < {table_name}.stop_sequence
                LIMIT 1
            ) AS previous_stop_id
        """,
        f"""
            (
                SELECT stop_name FROM pontos_stoptimes
                JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence < {table_name}.stop_sequence
                LIMIT 1
            ) AS previous_stop_name
        """,
        f"""
            (
                SELECT stop_id_id
                FROM pontos_stoptimes
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence > {table_name}.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_id
        """,
        f"""
            (
                SELECT stop_name FROM pontos_stoptimes
                JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence > {table_name}.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_name
        """,
        f"""
            (
                SELECT stop_sequence
                FROM pontos_stoptimes
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence < {table_name}.stop_sequence
                LIMIT 1
            ) AS previous_stop_sequence
        """,
        f"""
            (
                SELECT stop_sequence
                FROM pontos_stoptimes
                WHERE trip_id_id = {table_name}.trip_id_id AND stop_sequence > {table_name}.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_sequence
        """
    ]
    if return_type == "list":
        return ret
    elif return_type == "str":
        return ", ".join(ret)

def q_stops_prev_next(
    select=("*"),
    table=StopTimes._meta.db_table,
    conditions="",
    order_by=("trip_id_id", "stop_sequence")
):
    return f"""
    SELECT {", ".join(select)},
        {get_extra_stops_cols("st", "str")}
    FROM pontos_stoptimes AS st
    {conditions}
    ORDER BY {", ".join(order_by)}
    """
# replace first line
df_shapes = pd.read_sql_query(q_stops_prev_next()+" LIMIT 5", connection)
# df_shapes = pd.DataFrame.from_records(Shapes.objects.all().order_by("pk").values())
display(df_shapes)

C:\Users\Rapha\AppData\Local\Temp\ipykernel_34804\3247109099.py:76: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_shapes = pd.read_sql_query(q_stops_prev_next()+" LIMIT 5", connection)


,id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,stop_id_id,trip_id_id,previous_stop_id,previous_stop_name,next_stop_id,next_stop_name,previous_stop_sequence,next_stop_sequence
0,2337052,0,00:00:00,00:00:00,None,0,0,1,1,0.000000,None,5144O00042C0,0373fd52-e961-45ea-afb2-5959b62d11e3,None,None,5144O00046C0,Rua Lucília próximo ao 181-187,NaN,1
1,2337051,1,00:02:20,00:02:20,None,0,0,1,1,704.564858,None,5144O00046C0,0373fd52-e961-45ea-afb2-5959b62d11e3,5144O00042C0,Ponto Final: Linhas Zona Oeste :: via Rua Luc...,5144O00047C9,"Rua Lucília, lateral da FEUC.",0.0,2
2,2337050,2,00:02:32,00:02:32,None,0,0,1,1,764.610201,None,5144O00047C9,0373fd52-e961-45ea-afb2-5959b62d11e3,5144O00046C0,Rua Lucília próximo ao 181-187,5144O00048C9,"Estrada da Caroba, 878.",1.0,3
3,2337049,3,00:03:11,00:03:11,None,0,0,1,1,962.840505,None,5144O00048C9,0373fd52-e961-45ea-afb2-5959b62d11e3,5144O00047C9,"Rua Lucília, lateral da FEUC.",5144O00049C9,"Estrada das Capoeiras, 246.",2.0,4
4,2337048,4,00:05:09,00:05:09,None,0,0,1,1,1555.701025,None,5144O00049C9,0373fd52-e961-45ea-afb2-5959b62d11e3,5144O00048C9,"Estrada da Caroba, 878.",5144O00050C9,"Estrada das Capoeiras, 460.",3.0,5


### Simple test for ST_Distance

In [22]:
# create test table if not exists
cur.execute("""
CREATE TABLE IF NOT EXISTS test (
    id SERIAL PRIMARY KEY,
    lat float,
    lon float
)
""")
connection.commit()

# insert test data if not exists
cur.execute("""
INSERT INTO test (id, lat, lon)
VALUES (0, 1.0, 1.0), (1, 2.0, 2.0), (2, 3.0, 3.0)
ON CONFLICT DO NOTHING
""")
connection.commit()

# query test data
# q_test = "SELECT * FROM test"
# df_test = pd.read_sql_query(q_test, connection)
# display(df_test)

# clear all test data
# cur.execute("DELETE FROM test")
# connection.commit()

# show col distance3d using ST_Distance
q = """
SELECT *,
    ST_Distance(
        ST_MakePoint(lon, lat)::geography,
        ST_MakePoint(2.0, 2.0)::geography
    ) as distance3d
FROM test
"""
df_test = pd.read_sql_query(q, connection)
display(df_test)


C:\Users\Rapha\AppData\Local\Temp\ipykernel_28092\1225352124.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query(q, connection)


,id,lat,lon,distance3d
0,0,1.0,1.0,156876.149402
1,1,2.0,2.0,0.000000
2,2,3.0,3.0,156829.329116


### ⚠️ Teste para abrir popup do plot em nova janela do navegador
Não funciona no Jupyter do VSCode.

In [23]:
import pandas as pd
import numpy as np
df = pd.DataFrame(np.random.randn(6,4),columns=list('ABCD'))
# Show in Jupyter
df

from IPython.display import HTML
s  = '<script type="text/Javascript">'
s += 'var win = window.open("", "Title", "toolbar=no, location=no, directories=no, status=no, menubar=no, scrollbars=yes, resizable=yes, width=780, height=200, top="+(screen.height-400)+", left="+(screen.width-840));'
s += 'win.document.body.innerHTML = \'' + df.to_html().replace("\n",'\\') + '\';'
s += '</script>'

# Show in new Window
HTML(s)

### ✅ shapes with stops v3
Adicionando stop_next e stop_previous na versão 2.

In [25]:
# shapes_stops v3

q_shapes_stops_tst = f"""
SELECT 
    *
FROM (
    SELECT DISTINCT ON (id) * FROM (
        SELECT
            stoptimes.id as id,
            stoptimes.id as id__stoptimes,
            stoptimes.stop_sequence,
            stoptimes.shape_dist_traveled,
            stoptimes.previous_stop_id,
            stoptimes.next_stop_id,
            stoptimes.previous_stop_name,
            stoptimes.next_stop_name,
            routes.route_id,
            route_short_name,
            trip_id,
            trip_short_name,
            stops.stop_id,
            stop_name,
            stop_lat,
            stop_lon,
            shape_pt_lat,
            shape_pt_lon,
            route_long_name,
            shape_pt_sequence,
            shapes.shape_id,
            shapes.id as id__shapes,
            

            --CAST(SQRT(POW(CAST(shape_pt_lat AS DECIMAL(10,6)) - CAST(stop_lat AS DECIMAL(10,6)),2)
            -- + POW(CAST(shape_pt_lon AS DECIMAL(10,6)) - CAST(stop_lon AS DECIMAL(10,6)),2))
            --    AS DECIMAL(10,6)) AS distance

            ST_Distance(
                ST_MakePoint(shape_pt_lon, shape_pt_lat)::geography,
                ST_MakePoint(stop_lon, stop_lat)::geography
            ) AS distance

        FROM ({q_stops_prev_next()}) AS stoptimes
            JOIN pontos_trips trips ON (trips.trip_id = stoptimes.trip_id_id
            AND stoptimes.stop_id_id IN ('1001O00006C0', '1003O00006C0'))
            JOIN pontos_stops stops ON stoptimes.stop_id_id = stops.stop_id
            JOIN pontos_routes routes ON trips.route_id_id = routes.route_id
            JOIN pontos_shapes shapes ON trips.shape_id = shapes.shape_id
        ) as t1
    ORDER BY id, distance
) as t2


"""
# add cols previous_stop_id and next_stop_id from pontos_stoptimes
# use stop_sequence to get previous and next stop_id
# save to csv
df = pd.read_sql_query(q_shapes_stops_tst, connection)
# df.to_csv("df.csv", index=False)
q = q_shapes_stops_tst
# ju.print_query(q)
ju.plot_query(cur, q)

C:\Users\Rapha\AppData\Local\Temp\ipykernel_28092\3944564707.py:57: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q_shapes_stops_tst, connection)


len: 10


id,id__stoptimes,stop_sequence,shape_dist_traveled,previous_stop_id,next_stop_id,previous_stop_name,next_stop_name,route_id,route_short_name,trip_id,trip_short_name,stop_id,stop_name,stop_lat,stop_lon,shape_pt_lat,shape_pt_lon,route_long_name,shape_pt_sequence,shape_id,id__shapes,distance
2212190,2212190,3,594.0,1003O00035C0,1003O00016C0,Avenida Cidade de Lima - Santo Cristo,Via Binário do Porto próximo ao 742-778,O0104AAA0A,104,O0104AAA0AVDU03,104,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.897663,-43.202313,São Conrado - Rodoviária,18,O0104AAA0AVDU03,6499376,3.65995174
2221840,2221840,85,38083.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0384AAA0A,384,O0384AAA0AIDU02,384,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897331,-43.18731,Pavuna - Passeio,1537,O0384AAA0AIDU01,6650121,6.58062786
2234241,2234241,79,25374.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0265AAA0A,265,O0265AAA0AIDU01,265,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897329,-43.187309,Marechal Hermes - Castelo,1523,7ca9f20c-100a-41f7-9950-ead83d3e1031,6520425,6.6028241
2234437,2234437,69,27451.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0355AAA0A,355,O0355AAA0AIDU02,355,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.18731,Madureira - Praça Tiradentes,1412,O0355AAA0AIDU02,6617285,6.66212333
2240451,2240451,14,7817.0,1008O00001S0,1003O00016C0,BRS 4: Praça Onze,Via Binário do Porto próximo ao 742-778,E2145AAA0A,2145,E2145AAA0AVDU01,2145,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.89767,-43.20231,RIOgaleão - Aeroporto Santos Dumont,506,E2145AAA0AVDU01,6349992,3.69600084
2240923,2240923,11,16270.932695400263,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0041CAA0A,LECD41,O0041CAA0AIDU01,LECD41,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.187311,Vila do João - Alvorada,793,O0041CAA0AIDU01,7136203,6.56743519
2246369,2246369,13,18364.691150146442,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,E2018AAA0A,2018,E2018AAA0AIDU01,2018,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897332,-43.18731,RIOgaleão - Alvorada,897,E2018AAA0AIDU01,7095085,6.62057485
2246373,2246373,9,16265.240298304165,1003O00035C0,1003O00067C2,Avenida Cidade de Lima - Santo Cristo,Cidade do Samba,E2018AAA0A,2018,E2018AAA0AIDU01,2018,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.897663,-43.20231,RIOgaleão - Alvorada,713,E2018AAA0AIDU01,7094901,3.36786906
2248808,2248808,62,22890.872400628974,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0350AAA0A,350,O0350AAA0AIDU02,350,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897334,-43.187311,Irajá - Passeio,1373,O0350AAA0AIDU02,7087003,6.61117303
2249585,2249585,73,36595.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0385AAA0A,385,O0385AAA0AIDU01,385,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.18731,Village Pavuna - Passeio,1479,O0385AAA0AIDU01,7175953,6.66212333


### ✅ shapes with stops v4

Usando q_stops__in para obter as colunas extras de stops.

In [26]:
# shapes_stops v4

q_shapes_stops_tst = f"""
SELECT 
    *
FROM (
    SELECT DISTINCT ON (id) * FROM (
        SELECT
            stoptimes.id as id,
            stoptimes.id as id__stoptimes,
            stoptimes.stop_sequence,
            stoptimes.shape_dist_traveled,
            stoptimes.previous_stop_id,
            stoptimes.next_stop_id,
            stoptimes.previous_stop_name,
            stoptimes.next_stop_name,
            routes.route_id,
            route_short_name,
            trip_id,
            trip_short_name,
            stops.stop_id,
            stop_name,
            stop_lat,
            stop_lon,
            shape_pt_lat,
            shape_pt_lon,
            route_long_name,
            shape_pt_sequence,
            shapes.shape_id,
            shapes.id as id__shapes,

            --CAST(SQRT(POW(CAST(shape_pt_lat AS DECIMAL(10,6)) - CAST(stop_lat AS DECIMAL(10,6)),2)
            -- + POW(CAST(shape_pt_lon AS DECIMAL(10,6)) - CAST(stop_lon AS DECIMAL(10,6)),2))
            --    AS DECIMAL(10,6)) AS distance

            ST_Distance(
                ST_MakePoint(shape_pt_lon, shape_pt_lat)::geography,
                ST_MakePoint(stop_lon, stop_lat)::geography
            ) AS distance

        FROM ({q_stoptimes__in(
                stop_id=('1001O00006C0', '1003O00006C0'),
                query=q_stops_prev_next()
            )}) AS stoptimes
            JOIN pontos_trips trips ON trips.trip_id = stoptimes.trip_id_id
            JOIN pontos_stops stops ON stoptimes.stop_id_id = stops.stop_id
            JOIN pontos_routes routes ON trips.route_id_id = routes.route_id
            JOIN pontos_shapes shapes ON trips.shape_id = shapes.shape_id
        ) as t1
    ORDER BY id, distance
) as shapes_stoptimes
"""

# add cols previous_stop_id and next_stop_id from pontos_stoptimes
# use stop_sequence to get previous and next stop_id
# save to csv


# q_shapes_stops_tst = q = q_stoptimes__in(
#                 stop_id=('1001O00006C0', '1003O00006C0'),
#                 # select=('*', *get_extra_stops_cols('pontos_stoptimes'))
#                 query=q_stops_prev_next()
#             )

# ju.print_query(q_shapes_stops_tst)
df = pd.read_sql_query(q_shapes_stops_tst, connection)
# df.to_csv("df.csv", index=False)
q = q_shapes_stops_tst
ju.plot_query(cur, q)

C:\Users\Rapha\AppData\Local\Temp\ipykernel_28092\333685086.py:66: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q_shapes_stops_tst, connection)


len: 10


id,id__stoptimes,stop_sequence,shape_dist_traveled,previous_stop_id,next_stop_id,previous_stop_name,next_stop_name,route_id,route_short_name,trip_id,trip_short_name,stop_id,stop_name,stop_lat,stop_lon,shape_pt_lat,shape_pt_lon,route_long_name,shape_pt_sequence,shape_id,id__shapes,distance
2212190,2212190,3,594.0,1003O00035C0,1003O00016C0,Avenida Cidade de Lima - Santo Cristo,Via Binário do Porto próximo ao 742-778,O0104AAA0A,104,O0104AAA0AVDU03,104,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.897663,-43.202313,São Conrado - Rodoviária,18,O0104AAA0AVDU03,6499376,3.65995174
2221840,2221840,85,38083.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0384AAA0A,384,O0384AAA0AIDU02,384,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897331,-43.18731,Pavuna - Passeio,1537,O0384AAA0AIDU01,6650121,6.58062786
2234241,2234241,79,25374.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0265AAA0A,265,O0265AAA0AIDU01,265,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897329,-43.187309,Marechal Hermes - Castelo,1523,7ca9f20c-100a-41f7-9950-ead83d3e1031,6520425,6.6028241
2234437,2234437,69,27451.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0355AAA0A,355,O0355AAA0AIDU02,355,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.18731,Madureira - Praça Tiradentes,1412,O0355AAA0AIDU02,6617285,6.66212333
2240451,2240451,14,7817.0,1008O00001S0,1003O00016C0,BRS 4: Praça Onze,Via Binário do Porto próximo ao 742-778,E2145AAA0A,2145,E2145AAA0AVDU01,2145,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.89767,-43.20231,RIOgaleão - Aeroporto Santos Dumont,506,E2145AAA0AVDU01,6349992,3.69600084
2240923,2240923,11,16270.932695400263,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0041CAA0A,LECD41,O0041CAA0AIDU01,LECD41,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.187311,Vila do João - Alvorada,793,O0041CAA0AIDU01,7136203,6.56743519
2246369,2246369,13,18364.691150146442,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,E2018AAA0A,2018,E2018AAA0AIDU01,2018,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897332,-43.18731,RIOgaleão - Alvorada,897,E2018AAA0AIDU01,7095085,6.62057485
2246373,2246373,9,16265.240298304165,1003O00035C0,1003O00067C2,Avenida Cidade de Lima - Santo Cristo,Cidade do Samba,E2018AAA0A,2018,E2018AAA0AIDU01,2018,1003O00006C0,Avenida Professor Pereira Reis próximo ao 53-101,-22.897653,-43.202279,-22.897663,-43.20231,RIOgaleão - Alvorada,713,E2018AAA0AIDU01,7094901,3.36786906
2248808,2248808,62,22890.872400628974,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0350AAA0A,350,O0350AAA0AIDU02,350,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897334,-43.187311,Irajá - Passeio,1373,O0350AAA0AIDU02,7087003,6.61117303
2249585,2249585,73,36595.0,1002O00013C0,1002O00007C0,Hospital dos Servidores,Rua Camerino próximo ao 32,O0385AAA0A,385,O0385AAA0AIDU01,385,1001O00006C0,"Avenida Barão de Tefé, oposto ao 99",-22.89731,-43.18737,-22.897333,-43.18731,Village Pavuna - Passeio,1479,O0385AAA0AIDU01,7175953,6.66212333


### ✅ Function `shapes_stoptimes`

Add changes of shapes_with_stops based on shapes_with_stops function. ([see here](utils.py))

In [26]:
"""utils.py"""

def q_shapes_stoptimes(
    stop_id : list = (''),
    select_cols : list = ("*"),
):
    """
    v2.0 - 2023/01/06
    Get stoptimes with its related shape and join with stops, routes and trips

    Args:
        stop_id: list of stop_ids to filter
            Default: None
        select_cols: list of columns to select
            Default: "*"

    How it works:
        1. Get stoptimes, join with trips, stops, routes, shapes
            In stoptimes:
                Get previous and next stop_id and stop_name from stoptimes
                Filter by stop_id using criterias in q_stops_prev_next()
        2. Get distance between shape and stop
            If postgis is enabled, use 3D distance
            Else, use 2D distance
        3. For each id__stoptimes, get the shape with the smallest distance
            Using DISTINCT and ORDER BY
    """

    return f"""
SELECT
    {','.join(select_cols)}
FROM (
    SELECT DISTINCT ON (id__stoptimes) * FROM (
        SELECT
            *, 
            stoptimes.id as id,
            stoptimes.id as id__stoptimes,
            shapes.id as id__shapes,

            CASE
                WHEN EXISTS (SELECT PostGIS_full_version())
                THEN
                    ST_Distance(
                        ST_MakePoint(shape_pt_lon, shape_pt_lat)::geography,
                        ST_MakePoint(stop_lon, stop_lat)::geography
                    )
                ELSE
                    CAST(
                        SQRT(
                            POW(
                                CAST(shape_pt_lat AS DECIMAL(10,6))
                              - CAST(stop_lat AS DECIMAL(10,6))
                            ,2)
                            + POW(
                                CAST(shape_pt_lon AS DECIMAL(10,6))
                              - CAST(stop_lon AS DECIMAL(10,6))
                            ,2)
                        )
                    AS DECIMAL(10,6))
            END AS distance

        FROM ({q_stoptimes__in(
                stop_id=stop_id,
                query=q_stops_prev_next()
            )}) AS stoptimes
            JOIN pontos_trips trips ON trips.trip_id = stoptimes.trip_id_id
            JOIN pontos_stops stops ON stoptimes.stop_id_id = stops.stop_id
            JOIN pontos_routes routes ON trips.route_id_id = routes.route_id
            JOIN pontos_shapes shapes ON trips.shape_id = shapes.shape_id
        ) AS {qu.q_random_hash()}
    ORDER BY id__stoptimes, distance
) AS {qu.q_random_hash("shapes_stoptimes")}
    """

q = q_shapes_stoptimes(
    ['1001O00006C0', '1003O00006C0'],
    ["id__stoptimes as id"]
    )
# q += "LIMIT 10"
# df = pd.read_sql_query(q, connection)
# ju.print_query(q)
ju.plot_query(cur, q)


len: 10


id
2300587
2310237
2322638
2322834
2328848
2329320
2334766
2334770
2337205
2337982


Test stoptimes-plus query

Test `CASE WHEN (subquery)`

In [ ]:
# test CASE WHEN subquery
q = """
SELECT route_id,
    CASE
        WHEN EXISTS (SELECT PostGIS_full_version())
        THEN 'enabled'
        ELSE 'disabled'
    END AS postgis
FROM pontos_routes
ORDER BY route_id
LIMIT 10
"""
df = pd.read_sql_query(q, connection)
df

is_postgres_enabled()

In [ ]:
def is_postgis_enabled() -> bool:
    """Check if postgis is installed"""
    try:
        cur.execute("SELECT postgis_full_version()")
        return True
    except:
        connection.rollback()
        return False

enabled = is_postgis_enabled()
print(f"PostGIS is enabled: {enabled}")

PostGIS is enabled: True


get real fields

In [15]:
cols = Trips._meta.get_fields()
cols = [c.name for c in Trips._meta.get_fields()]
print(cols.index('trip_id'))

4


Solve bugs

In [40]:
# get list of cols


q = f"""
SELECT *
FROM (
    SELECT DISTINCT ON (id__stoptimes) * FROM (
        SELECT
            stoptimes.*,
            routes.*,
            trips.*,
            stops.*,
            shape_pt_lat,
            shape_pt_lon,
            shape_pt_sequence,
            shapes.shape_id,

            shapes.id AS id__shapes,
            stoptimes.id AS id__stoptimes,

            CASE
                WHEN EXISTS (SELECT PostGIS_full_version())
                THEN
                    ST_Distance(
                        ST_MakePoint(shape_pt_lon, shape_pt_lat)::geography,
                        ST_MakePoint(stop_lon, stop_lat)::geography
                    )
                ELSE
                    CAST(
                        SQRT(
                            POW(
                                CAST(shape_pt_lat AS DECIMAL(10,6))
                              - CAST(stop_lat AS DECIMAL(10,6))
                            ,2)
                            + POW(
                                CAST(shape_pt_lon AS DECIMAL(10,6))
                              - CAST(stop_lon AS DECIMAL(10,6))
                            ,2)
                        )
                    AS DECIMAL(10,6))
            END AS distance

        FROM (
                SELECT *
                FROM (
    SELECT *,

            (
                SELECT stop_id_id
                FROM pontos_stoptimes
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence < st.stop_sequence
                LIMIT 1
            ) AS previous_stop_id
        ,
            (
                SELECT stop_name FROM pontos_stoptimes
                JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence < st.stop_sequence
                LIMIT 1
            ) AS previous_stop_name
        ,
            (
                SELECT stop_id_id
                FROM pontos_stoptimes
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence > st.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_id
        ,
            (
                SELECT stop_name FROM pontos_stoptimes
                JOIN pontos_stops ON pontos_stoptimes.stop_id_id = pontos_stops.stop_id
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence > st.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_name
        ,
            (
                SELECT stop_sequence
                FROM pontos_stoptimes
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence < st.stop_sequence
                LIMIT 1
            ) AS previous_stop_sequence
        ,
            (
                SELECT stop_sequence
                FROM pontos_stoptimes
                WHERE trip_id_id = st.trip_id_id
                    AND stop_sequence > st.stop_sequence
                ORDER BY stop_sequence
                LIMIT 1
            ) AS next_stop_sequence

    FROM pontos_stoptimes AS st

    ORDER BY trip_id_id, stop_sequence
    ) AS q__0a2e7300be2a830925a04a678fd334a4
                WHERE stop_id_id IN ('1001O00006C0')
                ) AS stoptimes
            JOIN pontos_trips trips ON trips.trip_id = trip_id_id
            JOIN pontos_stops stops ON stoptimes.stop_id_id = stops.stop_id
            JOIN pontos_routes routes ON trips.route_id_id = routes.route_id
            JOIN pontos_shapes shapes ON trips.shape_id = shapes.shape_id
        ) AS q__7001cb0bedd7ea6f1c1039d5ff3c503e
    ORDER BY id__stoptimes, distance
) AS t
"""
# print(q)
df = pd.read_sql_query(q, connection)
# print all cols in list
cols = df.columns.tolist()
# cols = pd.DataFrame(cols)
print(cols)
# remove limit of df rows
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# df

C:\Users\Rapha\AppData\Local\Temp\ipykernel_34804\771757137.py:114: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(q, connection)


['id', 'stop_sequence', 'arrival_time', 'departure_time', 'stop_headsign', 'pickup_type', 'drop_off_type', 'continuous_pickup', 'continuous_drop_off', 'shape_dist_traveled', 'timepoint', 'stop_id_id', 'trip_id_id', 'previous_stop_id', 'previous_stop_name', 'next_stop_id', 'next_stop_name', 'previous_stop_sequence', 'next_stop_sequence', 'route_id', 'route_short_name', 'route_long_name', 'route_desc', 'route_type', 'route_url', 'route_branding_url', 'route_color', 'route_text_color', 'route_sort_order', 'continuous_pickup', 'continuous_drop_off', 'agency_id_id', 'service_id', 'trip_id', 'trip_headsign', 'trip_short_name', 'direction_id', 'block_id', 'shape_id', 'wheelchair_accessible', 'bikes_allowed', 'route_id_id', 'stop_id', 'stop_code', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon', 'zone_id', 'stop_url', 'location_type', 'stop_timezone', 'wheelchair_boarding', 'level_id', 'platform_code', 'parent_station_id', 'shape_pt_lat', 'shape_pt_lon', 'shape_pt_sequence', 'shape_id', 'id__